<a href="https://colab.research.google.com/github/Ritwik1607/Text-detection-app-using-got-ocr/blob/main/ritwik_Chandra_ocr_of_texts_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision torchaudio transformers tiktoken verovio accelerate


In [ ]:
!CUDA_VISIBLE_DEVICES=0 swift deploy --model_type llava1_6-vicuna-13b-instruct --infer_backend vllm

/bin/bash: line 1: swift: command not found


In [ ]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to upload files


Saving sample_image.jpg to sample_image.jpg


In [ ]:
!pip install modelscope swift -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.6/361.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 255
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyECLib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (PyECLib)


In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is available!


In [3]:
!pip install streamlit Pillow pyngrok transformers easyocr



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [6]:
!pip install streamlit Pillow pyngrok transformers

with open("app.py", "w") as f:
    f.write('''
import streamlit as st
from transformers import AutoModel, AutoTokenizer
import re
from PIL import Image
import io

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, use_safetensors=True, pad_token_id=tokenizer.eos_token_id)

# Set model to evaluation mode
model = model.eval().cuda()

# Streamlit application
st.title("OCR Application")
st.write("Upload an image to extract text in both English and Hindi.")

# Upload image file
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Convert the uploaded file to a format that can be processed by the model
    image = Image.open(uploaded_file)

    # Convert image to RGB if it's not
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image.save("temp_image.jpg", "JPEG")  # Save the uploaded image temporarily

    # Perform OCR using the model
    res = model.chat(tokenizer, "temp_image.jpg", ocr_type='ocr')

    # Text formatting
    res = re.sub(' +', ' ', res)  # Replace multiple spaces with a single space
    res = re.sub(r'(?<=[.,!?])(?=[^\s])', r' ', res)  # Add spaces after punctuation
    res = re.sub(r'(?<=[.!?]) +', r'\\g<0>\\n', res)  # Add line breaks after sentence endings

    st.subheader("Extracted Text:")

    # Search functionality
    search_term = st.text_input("Enter keywords to search within the extracted text:")

    if search_term:
        # Highlight the matching text
        highlighted_text = re.sub(f'({search_term})', r'<mark>\\1</mark>', res, flags=re.IGNORECASE)

        # Display search results with highlighted keywords
        st.markdown(f"<div style='white-space: pre-wrap;'>{highlighted_text}</div>", unsafe_allow_html=True)
    else:
        # Display extracted text without highlighting
        st.text(res)

''')

# Run the Streamlit app with ngrok
from pyngrok import ngrok

# Start the ngrok process
ngrok.set_auth_token("2mkhcc6oXNiJYBAchEpNiJg7mij_6jPQ6TgFL6QdvK33FbRen")  # Replace with your actual ngrok auth token
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

# Run the Streamlit app
!streamlit run app.py &


Public URL: NgrokTunnel: "https://e2a0-34-127-12-49.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.49:8501

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input